In [4]:
import pandas as pd
import numpy as np
import pickle, os

# Utils
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Models
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [11]:
DATA_PATH = 'data/cleaned_combined_gender.csv'
MODELS_PATH = 'models/bmi/'
RESULTS_PATH = 'results/bmi/'
os.makedirs(MODELS_PATH, exist_ok = True)
os.makedirs(RESULTS_PATH, exist_ok = True)

vars = pd.read_csv('variables.csv')
cat_cols = vars[vars['Variable Type'].fillna('-').str.contains('Cat')]['Variable Common Name'].values

In [12]:
df = pd.read_csv(DATA_PATH)

df = df.dropna(subset = ['BMI'])
assert df.isna().sum().sum() == 0

remove_cols = ['Hip Circumference (cm)', 'Waist Circumference (cm)', 'Weight (kg)']
X = df.drop(columns = ['BMI'] + remove_cols)
y = df['BMI']

cat_cols = [col for col in cat_cols if col in X.columns]
num_cols = [col for col in X.columns if col not in cat_cols]

X = pd.get_dummies(X, columns = cat_cols, drop_first = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [7]:
pipe_sel_from_model = lambda model, param_grid: GridSearchCV(

    make_pipeline(
        # Standardize numerical features
        ColumnTransformer(transformers = [('num', StandardScaler(), num_cols)], remainder = 'passthrough'),
        SelectFromModel(RandomForestRegressor(n_jobs = -1, random_state = 42)), # select most important features
        model
    ),

    # Search for the best no. of features to select and model hyperparameters
    param_grid = param_grid,

    # Use 5-fold cross validation
    cv = 5,

    # Refit the model with the best parameters on all the data
    # (will be stored in the best_estimator_ attribute)
    refit = True
)

In [8]:
max_features = (np.arange(0.25, 1.25, 0.25) * X_train.shape[1]).astype(int).tolist()
model_and_params = [
    (
        DummyRegressor(),
        {
            'selectfrommodel__max_features': max_features,
        }
    ),
    (
        DecisionTreeRegressor(),
        {
            'selectfrommodel__max_features': max_features,
            'decisiontreeregressor__max_depth': [None, 10, 100],
            # 'decisiontreeregressor__min_samples_split': [2, 4],
            # 'decisiontreeregressor__min_samples_leaf': [1, 2],
        }
    ),
    (
        RandomForestRegressor(n_jobs = -1),
        {
            'selectfrommodel__max_features': max_features,
            'randomforestregressor__n_estimators': [100, 500, 1000, 2000],
            'randomforestregressor__max_depth': [None, 10, 100],
            'randomforestregressor__min_samples_split': [2, 4],
            'randomforestregressor__min_samples_leaf': [1, 2],
        }
    ),
    (
        LinearRegression(),
        {
            'selectfrommodel__max_features': max_features,
        }
    ),
    (
        Lasso(),
        {
            'selectfrommodel__max_features': max_features,
            'lasso__alpha': [0.1, 0.5, 1, 5, 10],
        }
    ),
    (
        Ridge(),
        {
            'selectfrommodel__max_features': max_features,
            'ridge__alpha': [0.1, 0.5, 1, 5, 10],
        }
    ),
    (
        KNeighborsRegressor(),
        {
            'selectfrommodel__max_features': max_features,
            'kneighborsregressor__n_neighbors': [1, 3, 5, 10, 50, 100, 200],
        }
    ),
    (
        SVR(),
        {
            'selectfrommodel__max_features': max_features,
            'svr__C': [0.1, 0.5, 1, 5, 10],
            'svr__epsilon': [0.1, 0.5, 1, 5, 10],
        }
    ),
]

In [52]:
for model, param_grid in model_and_params:

    print(f'{model.__class__.__name__} ...')

    p = pipe_sel_from_model(model, param_grid)
    p.fit(X_train, y_train)
    print('Best params:', p.best_params_)

    # Save the GridSearchCV object
    with open(os.path.join(MODELS_PATH, f'{model.__class__.__name__}.pkl'), 'wb') as f:
        pickle.dump(p, f)

DummyRegressor ...
Best params: {'selectfrommodel__max_features': 30}
DecisionTreeRegressor ...
Best params: {'decisiontreeregressor__max_depth': 10, 'selectfrommodel__max_features': 30}
RandomForestRegressor ...
Best params: {'randomforestregressor__max_depth': 10, 'randomforestregressor__min_samples_leaf': 2, 'randomforestregressor__min_samples_split': 2, 'randomforestregressor__n_estimators': 100, 'selectfrommodel__max_features': 60}
LinearRegression ...
Best params: {'selectfrommodel__max_features': 30}
Lasso ...
Best params: {'lasso__alpha': 0.1, 'selectfrommodel__max_features': 30}
Ridge ...
Best params: {'ridge__alpha': 1, 'selectfrommodel__max_features': 30}
KNeighborsRegressor ...
Best params: {'kneighborsregressor__n_neighbors': 10, 'selectfrommodel__max_features': 30}
SVR ...
Best params: {'selectfrommodel__max_features': 30, 'svr__C': 10, 'svr__epsilon': 1}


In [13]:
results = []

for model_name in os.listdir(MODELS_PATH):

    # Load the GridSearchCV object
    with open(os.path.join(MODELS_PATH, model_name), 'rb') as f:
        grid_cv = pickle.load(f)

    # Columns selected by the SelectFromModel
    selected_cols = [c for c, s in zip(X_train.columns, grid_cv.best_estimator_[1].get_support()) if s]
    
    # Get the best score and parameters
    best_cv_score = grid_cv.best_score_
    best_params = grid_cv.best_params_
    preds = grid_cv.best_estimator_[-1].predict(StandardScaler().fit_transform(X_test[selected_cols]))
    test_score = r2_score(y_test, preds)
    selected_features = grid_cv.best_estimator_.named_steps['selectfrommodel'].get_support().sum()

    results.append((model_name.split('.')[0], best_cv_score, test_score, selected_features))

results = pd.DataFrame(results, columns = ['Model', 'Best CV Score', 'Test Score', 'No. of selected vars']).sort_values('Test Score', ascending = False)
results.to_csv(os.path.join(RESULTS_PATH, 'results_bmi.csv'), index = False)
results

,Model,Best CV Score,Test Score,No. of selected vars
0,RandomForestRegressor,0.824425,0.838371,9
4,Lasso,0.829501,0.812252,9
6,KNeighborsRegressor,0.775811,0.780703,9
5,Ridge,0.831287,0.779718,9
2,LinearRegression,0.831286,0.779266,9
7,SVR,0.824601,0.768749,9
1,DecisionTreeRegressor,0.696182,0.697705,9
3,DummyRegressor,-0.002128,-0.000089,9


In [14]:
# Load the best GridSearchCV object (random forest)
with open(os.path.join(MODELS_PATH, 'RandomForestRegressor' + '.pkl'), 'rb') as f:
    grid_cv = pickle.load(f)

selected = [c for c, s in zip(X_train.columns, grid_cv.best_estimator_[1].get_support()) if s]
imps = pd.Series(grid_cv.best_estimator_[-1].feature_importances_, index = selected).sort_values(ascending = False)
imps.to_csv(os.path.join(RESULTS_PATH, 'feature_importances_bmi.csv'), header = False)
imps

Arm Circumference (cm)                      0.810384
Insulin (μU/mL)                             0.040876
Gender_2.0                                  0.031159
Doctor ever said you were overweight_2.0    0.029190
Age in years                                0.021178
Total Cholesterol (mg/dL)                   0.017401
Triglyceride (mg/dL)                        0.017304
Glycohemoglobin (%)  (AIC)                  0.016923
Direct HDL-Cholesterol (mg/dL)              0.015587
dtype: float64